In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,965 kB]
Get:14 http://archive.ubuntu.

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName("RDD").getOrCreate()
spark

In [4]:
!wget https://raw.githubusercontent.com/Bugg-Bitter/BiG_data_IOT/main/Lab%20-%204/evan.txt


--2022-09-16 16:45:31--  https://raw.githubusercontent.com/Bugg-Bitter/BiG_data_IOT/main/Lab%20-%204/evan.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2144 (2.1K) [text/plain]
Saving to: ‘evan.txt’

evan.txt            100%[===================>]   2.09K  --.-KB/s    in 0s      

2022-09-16 16:45:31 (39.0 MB/s) - ‘evan.txt’ saved [2144/2144]



In [5]:
saved = spark.sparkContext.textFile("/content/evan.txt")
saved.collect()

['Contrary to popular belief, Lorem Ipsum is not simply random text. It has roots in a piece of ',
 'classical Latin literature from 45 BC, making it over 2000 years old. Richard McClintock, a ',
 'Latin professor at Hampden-Sydney College in Virginia, looked up one of the more obscure Latin ',
 'words, consectetur, from a Lorem Ipsum passage, and going through the cites of the word in ',
 'classical literature, discovered the undoubtable source. Lorem Ipsum comes from sections ',
 '1.10.32 and 1.10.33 of "de Finibus Bonorum et Malorum" (The Extremes of Good and Evil) by ',
 'Cicero, written in 45 BC. This book is a treatise on the theory of ethics, very popular during ',
 'the Renaissance. The first line of Lorem Ipsum, "Lorem ipsum dolor sit amet..", comes from a ',
 'line in section 1.10.32.',
 '',
 'The standard chunk of Lorem Ipsum used since the 1500s is reproduced below for those ',
 'interested. Sections 1.10.32 and 1.10.33 from "de Finibus Bonorum et Malorum" by Cicero are ',


In [6]:
splits = saved.flatMap(lambda x: x.split(" "))
splits.take(24)

['Contrary',
 'to',
 'popular',
 'belief,',
 'Lorem',
 'Ipsum',
 'is',
 'not',
 'simply',
 'random',
 'text.',
 'It',
 'has',
 'roots',
 'in',
 'a',
 'piece',
 'of',
 '',
 'classical',
 'Latin',
 'literature',
 'from',
 '45']

In [7]:
pairs = splits.map(lambda x: (x,3))
pairs.take(8)

[('Contrary', 3),
 ('to', 3),
 ('popular', 3),
 ('belief,', 3),
 ('Lorem', 3),
 ('Ipsum', 3),
 ('is', 3),
 ('not', 3)]

In [8]:
rdByKey = pairs.reduceByKey(lambda a,b:a*b)
rdByKey.take(8)

[('Contrary', 9),
 ('to', 9),
 ('popular', 81),
 ('belief,', 9),
 ('Lorem', 59049),
 ('Ipsum', 6561),
 ('is', 729),
 ('not', 9)]

In [9]:
sorted = rdByKey.map(lambda x: (x[0], x[1])).sortByKey()
sorted.take(8)

[('', 282429536481),
 ('"Lorem', 9),
 ('"de', 81),
 ('(The', 9),
 ('1.10.32', 81),
 ('1.10.32.', 9),
 ('1.10.33', 81),
 ('1500s', 9)]

In [10]:
#id: 193-15-2948
totalPartitions = sorted.repartition(8) 
print("Number of Partitions: "+ str(totalPartitions.getNumPartitions()))

Number of Partitions: 8


In [13]:
totalPartitions.saveAsTextFile("/content/saved")